<a href="https://colab.research.google.com/github/gudiseva/AI-Workshop/blob/main/CO2_Emissions_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Topic: CO2 Emissions

### Importing Required Libraries 

In [1]:
# Essential 
import numpy as np 
import pandas as pd 

# Visualization
import matplotlib.pyplot as plt
import seaborn as sb

# encoding 
from sklearn.preprocessing import LabelEncoder

# Normalization
from sklearn.preprocessing import PolynomialFeatures

# Model Building 
import sklearn
from sklearn.model_selection import train_test_split


from sklearn.linear_model import LinearRegression as LR
from xgboost import XGBRegressor as XGB 
from sklearn.ensemble import AdaBoostRegressor as Ada
from sklearn.tree import DecisionTreeRegressor as DT
from sklearn.ensemble import RandomForestRegressor as RF
from sklearn.ensemble import GradientBoostingRegressor as GB

# Evaluation 
from sklearn.metrics import r2_score,mean_squared_error

# Cross Validation 
from sklearn.model_selection import GridSearchCV

In [2]:
#Customised function

def missing_cols(df):
    '''prints out columns with its amount of missing values with its %'''
    total = 0
    for col in df.columns:
        missing_vals = df[col].isnull().sum()
        pct = df[col].isna().mean() * 100
        total += missing_vals
        if missing_vals != 0:
          print('{} => {} [{}%]'.format(col, df[col].isnull().sum(), round(pct, 2)))
    
    if total == 0:
        print("no missing values")

### Uploading Training Data

In [4]:
#df = pd.read_csv('train.csv')
df = pd.read_csv('emission_training_data.csv')
df

,MODELYEAR,MAKE,MODEL,VEHICLECLASS,ENGINESIZE,CYLINDERS,TRANSMISSION,FUELTYPE,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS
0,2011,HONDA,ACCORD 4DR SEDAN,FULL-SIZE,3.5,6,A5,X,11.7,7.8,9.9,29,228
1,2011,SCION,tC,COMPACT,2.5,4,AS6,X,10.2,7.5,9.0,31,207
2,2011,ROLLS-ROYCE,PHANTOM EWB,FULL-SIZE,6.7,12,A6,Z,20.6,13.4,17.4,16,400
3,2011,CHEVROLET,COLORADO,PICKUP TRUCK - SMALL,2.9,4,A4,X,12.8,9.6,11.3,25,260
4,2011,MAZDA,MAZDA2,COMPACT,1.5,4,M5,X,8.0,6.7,7.4,38,170
...,...,...,...,...,...,...,...,...,...,...,...,...,...
754,2011,MERCEDES-BENZ,G 55 AMG,SUV,5.4,8,A5,Z,21.8,17.3,19.8,14,455
755,2011,NISSAN,TITAN,PICKUP TRUCK - STANDARD,5.6,8,A5,X,17.9,12.8,15.6,18,359
756,2011,BMW,M3 CABRIOLET,SUBCOMPACT,4.0,8,M6,Z,17.4,11.9,14.9,19,343
757,2011,CHEVROLET,HHR PANEL,SUV,2.2,4,M5,E,14.2,10.2,12.4,23,198


### Uploading Testing Data

In [5]:
#df2 = pd.read_csv('test.csv')
df2 = pd.read_csv('emissions_testing_data.csv')
df2

,MODELYEAR,MAKE,MODEL,VEHICLECLASS,ENGINESIZE,CYLINDERS,TRANSMISSION,FUELTYPE,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG
0,2011,HONDA,PILOT AWD,SUV,3.5,6,A5,X,14.7,10.7,12.9,22
1,2011,CADILLAC,CTS AWD,MID-SIZE,3.6,6,AS6,X,13.2,8.8,11.2,25
2,2011,DODGE,JOURNEY AWD,SUV,3.6,6,AS6,X,14.6,9.9,12.5,23
3,2011,BMW,323i,COMPACT,2.5,6,M6,Z,12.6,8.2,10.6,27
4,2011,GMC,YUKON,SUV,5.3,8,A6,E,21.5,14.7,18.5,15
...,...,...,...,...,...,...,...,...,...,...,...,...
321,2011,NISSAN,GT-R,SUBCOMPACT,3.8,6,AM6,Z,15.5,11.2,13.6,21
322,2011,MERCEDES-BENZ,SLS AMG,TWO-SEATER,6.2,8,AS7,Z,17.3,12.1,15.0,19
323,2011,VOLKSWAGEN,JETTA,COMPACT,2.0,4,AS6,X,10.6,8.0,9.5,30
324,2011,SUBARU,IMPREZA WAGON AWD,STATION WAGON - SMALL,2.5,4,M5,X,12.2,8.8,10.7,26


## EDA

### EDA (Exploratory Data Analysis) is a data analysis technique by which we try to understand the given data and the relationship among those data and also try to find out features , insights and anomalies from the data using statistics and graphical representation.

In [ ]:
df = df.drop_duplicates()
df

In [ ]:
df.describe()

In [ ]:
df.nunique()

In [ ]:
df['FUELTYPE'].value_counts()

In [ ]:
sb.pairplot(df)

In [ ]:
sb.heatmap(df.corr(),annot = True, cmap = 'magma')

In [ ]:
sb.distplot(df['CO2EMISSIONS'], kde = False)

In [ ]:
sb.distplot(df['CYLINDERS'], kde = False)

In [ ]:
plt.figure(figsize = (14,7))
sb.countplot( x = 'MAKE', data = df)
plt.show()

#plt.grid()

In [ ]:
plt.figure(figsize = (16,8))
plt.title("Box plot for CO2 Emissions")
sb.boxplot( y = 'CO2EMISSIONS', x = 'CYLINDERS', data = df)
plt.show()

In [ ]:
plt.figure(figsize = (16,8))
plt.title("Scatter plot for CO2 Emissions")
plt.scatter(y = 'CO2EMISSIONS', x = 'CYLINDERS', data = df)

In [ ]:
plt.figure(figsize = (16,8))
plt.title("Box plot for CO2 Emissions")
sb.boxplot( y = 'CO2EMISSIONS', x = 'FUELTYPE', data = df)
plt.show()

In [ ]:
plt.figure(figsize = (16,8))
plt.title("Scatter plot for CO2 Emissions")
plt.scatter(y = 'CO2EMISSIONS', x = 'FUELCONSUMPTION_CITY', data = df)

In [ ]:
plt.figure(figsize=(16,10))
ax = sb.scatterplot(x='FUELCONSUMPTION_CITY',y='CO2EMISSIONS',data=df, hue = 'FUELTYPE', palette='rainbow')
ax.set_title('Scatter plot of CO2 Emissions')

In [ ]:
plt.figure(figsize=(16,10))
ax = sb.scatterplot(x='FUELCONSUMPTION_CITY',y='CO2EMISSIONS',data=df, hue = 'CYLINDERS', palette='rainbow')
ax.set_title('Scatter plot of CO2 Emissions')

In [ ]:
plt.figure(figsize=(16,10))
ax = sb.scatterplot(x='FUELCONSUMPTION_HWY',y='CO2EMISSIONS',data=df, hue = 'FUELTYPE', palette='rainbow')
ax.set_title('Scatter plot of CO2 Emissions')

In [ ]:
plt.figure(figsize=(16,10))
ax = sb.scatterplot(x='FUELCONSUMPTION_COMB',y='CO2EMISSIONS',data=df, hue = 'FUELTYPE', palette='rainbow')
ax.set_title('Scatter plot of CO2 Emissions')

In [ ]:
plt.figure(figsize=(16,10))
ax = sb.scatterplot(x='FUELCONSUMPTION_COMB_MPG',y='CO2EMISSIONS',data=df, hue = 'FUELTYPE', palette='rainbow')
ax.set_title('Scatter plot of CO2 Emissions')

### Preprocessing on Training Data

In [ ]:
missing_cols(df)

In [ ]:
df.nunique()

In [ ]:
df.info()

In [ ]:
objList = df.select_dtypes(include = "object").columns
print (objList)

In [ ]:
#Label Encoding for object to numeric conversion
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for feat in objList:
    df[feat] = le.fit_transform(df[feat].astype(str))

print (df.info())

In [ ]:
df

### Preprocessing on Testing Data

In [ ]:
objList = df2.select_dtypes(include = "object").columns
print (objList)

In [ ]:
#Label Encoding for object to numeric conversion
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for feat in objList:
    df2[feat] = le.fit_transform(df2[feat].astype(str))

print (df2.info())

### xVar & yVar

In [ ]:
xVar = df.drop(['CO2EMISSIONS'], axis = 1)
xVar

In [ ]:
yVar = df['CO2EMISSIONS']
yVar

### Train-test Split

In [ ]:
#data split

X_train, X_test, y_train, y_test = train_test_split(xVar, yVar, test_size=0.25, random_state = 0)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

In [ ]:
#test data 

X_test = df2
X_test

In [ ]:
#test data

X_train = xVar
y_train = yVar
print (X_train.shape, y_train.shape)
print (X_test.shape)

## Scaling and Normalization

### Reference: 
https://machinelearningmastery.com/polynomial-features-transforms-for-machine-learning/

In [ ]:
## Scaling ## Not performed for this dataset 

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# transform data
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [ ]:
## PolynomialFeatures improves accuracy of Linear regression but not necessarily of decision tree based algorithms 

quad = PolynomialFeatures (degree = 2)

X_train = quad.fit_transform(X_train)
X_test = quad.fit_transform(X_test)

## Simple Linear Regression

### Reference:
https://www.youtube.com/watch?app=desktop&v=U7D1h5bbpcs&feature=youtu.be

In [ ]:
import statsmodels.api as sm

X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)


model = sm.OLS(y_train, X_train)
model = model.fit()
print(model.summary2())

In [ ]:
predictions = model.predict(X_test)
df_results = pd.DataFrame({'Actual': y_test, 'Predicted': predictions})

In [ ]:
RMSE = np.sqrt(mean_squared_error(y_test, predictions))
r2 = r2_score(y_test, predictions)
print(RMSE, r2)

In [ ]:
# Plot the actual vs predicted results
plt.figure(figsize=(16,10))
sb.lmplot(x='Actual', y='Predicted', data=df_results, fit_reg=False)

#Plot the diagonal line
d_line= np.arange(df_results.min().min(), df_results.max().max())
plt.plot(d_line, d_line, color='red', linestyle='--')
plt.show()

## Different Algorithms 

In [ ]:
# Implementing models

# Linear Regression
linear = LR()
lr = linear.fit(X_train, y_train)
lr.score(X_test,y_test)

In [ ]:
# Adaboost Regressor

ada = Ada(random_state=0)

ada_ = ada.fit(X_train, y_train)

ada_.score(X_test,y_test)

In [ ]:
# XGB Regressor

xgbregressor = XGB(random_state=0)

xgb_ = xgbregressor.fit(X_train, y_train)

xgb_.score(X_test,y_test)

In [ ]:
# Decision Tree Regressor

dtr = DT(criterion="mse",
        max_depth=6,
        max_features="auto",
        random_state=0)

dtr_ = dtr.fit(X_train, y_train)

dtr_.score(X_test,y_test)

In [ ]:
# Random Tree Regressor

rfr = RF(n_estimators=100,
        criterion="mse",
        max_depth=6,
        max_features="auto",
        random_state=0)

rfr_ = rfr.fit(X_train, y_train)

rfr_.score(X_test,y_test)

## How to check if model is overfitting or not 

In [ ]:
forest = RF(n_estimators = 500,
            criterion = 'mse',
            random_state = 0,
            n_jobs = -1)
forest.fit(X_train,y_train)
forest_train_pred = forest.predict(X_train)
forest_test_pred = forest.predict(X_test)

print('MSE train data: %.3f, MSE test data: %.3f' % (
mean_squared_error(y_train,forest_train_pred),
mean_squared_error(y_test,forest_test_pred)))

print('R2 train data: %.3f, R2 test data: %.3f' % (
r2_score(y_train,forest_train_pred),
r2_score(y_test,forest_test_pred)))

### Validating Models

In [ ]:
lr_pred = lr.predict(X_test)
ada_pred = ada_.predict(X_test)
xgb_pred = xgb_.predict(X_test)
dtr_pred = dtr_.predict(X_test)
rfr_pred = rfr_.predict(X_test)

In [ ]:
# Plot the actual vs predicted results
plt.figure(figsize=(16,16))
df_results = pd.DataFrame({'Actual': y_test, 'Predicted': lr_pred})

sb.lmplot(x='Actual', y='Predicted', data=df_results, fit_reg=False)

d_line= np.arange(df_results.min().min(), df_results.max().max())
plt.plot(d_line, d_line, color='red', linestyle='--')
plt.show()

In [ ]:
# Plot the actual vs predicted results
plt.figure(figsize=(16,16))
df_results = pd.DataFrame({'Actual': y_test, 'Predicted': xgb_pred})

sb.lmplot(x='Actual', y='Predicted', data=df_results, fit_reg=False)

d_line= np.arange(df_results.min().min(), df_results.max().max())
plt.plot(d_line, d_line, color='red', linestyle='--')
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
linear_mse = mean_squared_error(y_test, lr_pred)
ada_mse = mean_squared_error(y_test, ada_pred)
xgb_mse = mean_squared_error(y_test, xgb_pred)
dtr_mse = mean_squared_error(y_test, dtr_pred)
rfr_mse = mean_squared_error(y_test, rfr_pred)

In [ ]:
print(pow(linear_mse, 0.5))
print(pow(ada_mse, 0.5))
print(pow(xgb_mse, 0.5))
print(pow(dtr_mse, 0.5))
print(pow(rfr_mse, 0.5))

## Let's try to Improve the Model 

### XGBoost Model 

In [ ]:
## Note: It may take a few minutes. 

xgb1 = XGBR()
parameters = {'nthread':[4], 
              'objective':['reg:squarederror'],
              'learning_rate': [0.01, 0.03, 0.1],
              'max_depth': [5, 6, 7],
              'min_child_weight': [1, 5],
              'subsample': [0.6, 0.8, 1.0],
              'colsample_bytree': [0.6, 0.8, 1.0],
              'n_estimators': [500],
              'gamma': [0.5, 1, 1.5, 2, 5]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 3,
                        n_jobs = 5,
                        scoring='mean_squared_error',
                        verbose=True)

xgb_grid.fit(X_train,
         y_train)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

In [ ]:
xgboost = XGB(learning_rate=0.04,
                       nthread = 4,
                       n_estimators=500,
                       max_depth=5,
                       min_child_weight=1,
                       gamma=1.5,
                       objective='reg:squarederror',
                       subsample= 0.8,
                       random_state=42)

xgb = xgboost.fit(X_train, y_train)

### Prediction 

In [ ]:
prediction = xgb.predict(X_test)
prediction

### RMSE Calculation for training data

In [ ]:
from sklearn.metrics import mean_squared_error
RMSE = np.sqrt(mean_squared_error(y_test, prediction))
print(RMSE)

In [ ]:
# Plot the actual vs predicted results
plt.figure(figsize=(16,16))
df_results = pd.DataFrame({'Actual': y_test, 'Predicted': prediction})

sb.lmplot(x='Actual', y='Predicted', data=df_results, fit_reg=False)

d_line= np.arange(df_results.min().min(), df_results.max().max())
plt.plot(d_line, d_line, color='red', linestyle='--')
plt.show()

## Decision Tree Regressor 

### Convert into csv 

In [ ]:
from pandas import DataFrame
df3 = DataFrame(prediction,columns=['CO2EMISSIONS'], index = False)

In [ ]:
df3.to_csv('submission.csv')

# End of code